In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares, minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
from tqdm import tqdm

def f(x, a):
    return (a[1] * x + a[0]) / (x + a[2])

N = 10**2
len_x = 30
a_origin = np.array([0.0, 1.0, 1.0])
x = np.linspace(0.1, 3.0, len_x)
std = 0.05 

y_normals = []

for i in (range(N)):
    eps_normal = np.random.normal(0, std, len_x)
    y_normal = f(x, a_origin) + eps_normal
    y_normals.append(y_normal)

from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

def ols_loss(a, x, y):
    return np.sum((f(x, a) - y) ** 2)

def lar_loss(a, x, y):
    return np.sum(np.abs(f(x, a) - y))

methods = {
    "OLS": ols_loss,
    "LAR": lar_loss,
}

def regression(method, x, y, a_origin):
    loss_function = methods[method]
    result = minimize(loss_function, a_origin, args=(x, y))
    return result.x

def add_outliers(y, k, q):
    outlier_quantiles = [0.95, 0.99, 0.999]
    y_with_outliers = y.copy()

    q95 = np.percentile(y, outlier_quantiles[0] * 100)
    q99 = np.percentile(y, outlier_quantiles[1] * 100)
    q999 = np.percentile(y, outlier_quantiles[2] * 100)

    outlier_indices = np.random.choice(len(y), size=k, replace=False)
    
    for i in outlier_indices:
        if q == 1:
            y_with_outliers[i] = q95
        elif q == 2:
            y_with_outliers[i] = q99
        elif q == 3:
            y_with_outliers[i] = q999

    return y_with_outliers


k_values = [1, 2, 3]
quantile_indexes = [1, 2, 3]
quantile_names = ['95%', '99%', '99.9%']

for q in quantile_indexes:
    print('\n', quantile_names[q-1], ' квантиль')
    for k in (k_values):
        print(k, ' выбросов')

        outlier_counts = {"OLS": 0, "LAR": 0}

        for i in (range(N)):
            y_normal_with_outliers = add_outliers(y_normals[i], k, q)
    
            for method in methods:
                    results = []

            for method in methods:
                a = regression(method, x, y_normal_with_outliers, a_origin)
                quantiles = [
                    norm.ppf(0.95, loc=0, scale=std),
                    norm.ppf(0.99, loc=0, scale=std),
                    norm.ppf(0.999, loc=0, scale=std)
                ]
                if np.any(np.abs(a - a_origin) > quantiles[q-1]):
                    outlier_counts[method] += 1
        for method in methods:     
            print(f"Метод {method}: {outlier_counts[method] / N  * 100:.2f}% случаев, когда хотя бы один выброс остается")


 95%  квантиль
1  выбросов
Метод OLS: 86.00% случаев, когда хотя бы один выброс остается
Метод LAR: 60.00% случаев, когда хотя бы один выброс остается
2  выбросов
Метод OLS: 86.00% случаев, когда хотя бы один выброс остается
Метод LAR: 61.00% случаев, когда хотя бы один выброс остается
3  выбросов
Метод OLS: 95.00% случаев, когда хотя бы один выброс остается
Метод LAR: 69.00% случаев, когда хотя бы один выброс остается

 99%  квантиль
1  выбросов
Метод OLS: 74.00% случаев, когда хотя бы один выброс остается
Метод LAR: 62.00% случаев, когда хотя бы один выброс остается
2  выбросов
Метод OLS: 85.00% случаев, когда хотя бы один выброс остается
Метод LAR: 57.00% случаев, когда хотя бы один выброс остается
3  выбросов
Метод OLS: 85.00% случаев, когда хотя бы один выброс остается
Метод LAR: 58.00% случаев, когда хотя бы один выброс остается

 99.9%  квантиль
1  выбросов
Метод OLS: 72.00% случаев, когда хотя бы один выброс остается
Метод LAR: 47.00% случаев, когда хотя бы один выброс остаетс